In [1]:
import torchxrayvision as xrv

In [2]:
!git clone https://github.com/ieee8023/covid-chestxray-dataset

Cloning into 'covid-chestxray-dataset'...
remote: Enumerating objects: 3641, done.
remote: Total 3641 (delta 0), reused 0 (delta 0), pack-reused 3641
Receiving objects: 100% (3641/3641), 632.96 MiB | 11.97 MiB/s, done.
Resolving deltas: 100% (1450/1450), done.
Updating files: 100% (1174/1174), done.


In [3]:
d = xrv.datasets.COVID19_Dataset(imgpath="covid-chestxray-dataset/images/",csvpath="covid-chestxray-dataset/metadata.csv")

In [4]:
import matplotlib.pyplot as plt

In [12]:
d.csv.head()

,index,patientid,offset,sex,age,finding,RT_PCR_positive,survival,intubated,intubation_present,...,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 29,offset_day_int
0,0,2,0.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,...,"Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN,0.0
1,1,2,3.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,...,"Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN,3.0
2,2,2,5.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,...,"Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN,5.0
3,3,2,6.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,...,"Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN,6.0
4,4,4,0.0,F,52.0,Pneumonia/Viral/COVID-19,Y,NaN,N,N,...,"Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN,NaN,0.0


In [22]:
split_tags = [ val.split('/') for val in d.csv['finding'] ]

In [28]:
all_tags = set()

for split_tag in split_tags:
    for val in split_tag:
        all_tags.add(val)

all_tags = list(all_tags)
print(all_tags)

['Bacterial', 'Klebsiella', 'H1N1', 'SARS', 'Aspergillosis', 'Staphylococcus', 'Herpes ', 'Varicella', 'Viral', 'Pneumonia', 'Mycoplasma', 'Streptococcus', 'Nocardia', 'MERS-CoV', 'MRSA', 'Influenza', 'Lipoid', 'Pneumocystis', 'No Finding', 'Chlamydophila', 'Fungal', 'Aspiration', 'Legionella', 'Tuberculosis', 'COVID-19']


In [36]:
tag_index = {'Bacterial':1, 'Klebsiella':1, 'H1N1':2, 'SARS':0, 'Aspergillosis':1, 'Staphylococcus':1, 'Herpes ':0, 'Varicella':0, 'Viral':0, 'Pneumonia':0, 'Mycoplasma':1, 'Streptococcus':1, 'Nocardia':1, 'MERS-CoV':0, 'MRSA':2, 'Influenza':0, 'Lipoid':2, 'Pneumocystis':1, 'No Finding':2, 'Chlamydophila':1, 'Fungal':1, 'Aspiration':2, 'Legionella':1, 'Tuberculosis':2, 'COVID-19':0}

In [37]:
data = d.csv[['finding', 'filename']]

In [38]:
data.head()

,finding,filename
0,Pneumonia/Viral/COVID-19,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...
1,Pneumonia/Viral/COVID-19,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...
2,Pneumonia/Viral/COVID-19,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...
3,Pneumonia/Viral/COVID-19,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...
4,Pneumonia/Viral/COVID-19,nejmc2001573_f1a.jpeg


In [40]:
bitmasks = []

for i, row in data.iterrows():
    bitmask = 0
    
    for val in [tag_index[extracted_tag] for extracted_tag in row['finding'].split('/') ]:
        bitmask = bitmask | (1 << val)

    bitmasks.append(bitmask)

data['bitmasks'] = bitmasks
data.head()
    

,finding,filename,bitmasks
0,Pneumonia/Viral/COVID-19,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,1
1,Pneumonia/Viral/COVID-19,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,1
2,Pneumonia/Viral/COVID-19,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,1
3,Pneumonia/Viral/COVID-19,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,1
4,Pneumonia/Viral/COVID-19,nejmc2001573_f1a.jpeg,1


In [41]:
print(data['bitmasks'].unique())

[1 3 4 5 7]
